In [50]:
import pandas as pd
import numpy as np
import mysql # Para ver la versión
import mysql.connector
import pymysql

**CREAR BASE DE DATOS**

In [53]:

database = "AEMET"

def crear_database(database, host = "localhost", user = "root", password = "password"):

    # Inicializar Connector
    db = mysql.connector.connect(host     = "localhost",
                                 user     = "root",
                                 password = "nueva_contraseña",
                                 database = None)
    
    # Iniciar Cursor
    cursor = db.cursor()
    
    # Creamos la BBDD con la setencia de SQL y usando la variable cursor
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")
    
    # Finalizar Cursor - Cierra el cursor
    cursor.close()
    
    # Finalizar Connector
    db.close()

In [55]:
crear_database(database, host = "localhost", user = "root", password = "password")

**CREAR TABLAS DENTRO DE LA BASE DE DATOS**

In [58]:
database = "AEMET"

def execute_query(query, database, host = "localhost", user = "root", password = "password"):
    
    db = mysql.connector.connect(host     = "localhost",
                                 user     = "root",
                                 password = "nueva_contraseña",
                                 database = database)
    
    cursor = db.cursor()
    
    cursor.execute(query)
    
    cursor.fetchall() # Vaciamos el cursor
    cursor.close()
    db.close()

In [60]:
with open("nombre.txt", "r") as file:
    query = file.read()

In [62]:
execute_query(query = query, database = database)

In [64]:
with open("provincia.txt", "r") as file:
    query = file.read()

In [66]:
execute_query(query = query, database = database)

In [68]:
with open("indicativo.txt", "r") as file:
    query = file.read()

In [70]:
execute_query(query = query, database = database)

In [72]:
with open("valores_climatologicos.txt", "r") as file:
    query = file.read()

In [74]:
execute_query(query = query, database = database)

**INSERTAR DATOS**

In [77]:
def insert_to_table(data, table, database, host = "localhost", user = "root", password = "nueva_contraseña"):
    
    db = mysql.connector.connect(host     = host,
                                 user     = user,
                                 password = password,
                                 database = database)
    cursor = db.cursor()

    # Seleccionamos las columnas de la tabla, omitiendo la Primary Key
    cursor.execute(f"SELECT * FROM {table} LIMIT 0;")
    column_names = cursor.column_names[1:]
    cursor.fetchall()

    insert_query = f"INSERT INTO {table} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})".replace("'", "")
    values = [tuple(row) for row in data]

    # .executemany ejecuta el query de INSERT INTO con cada uno de los elementos de "values"
    cursor.executemany(insert_query, values)
    
    # Guarda los resultados
    db.commit()

    print(f"Añadidas: {cursor.rowcount} filas")

    cursor.fetchall() # Vaciamos el cursor
    cursor.close()
    db.close()

In [79]:
data = pd.read_csv("nombre.csv", index_col=0).values
table = "nombre"
database = "AEMET"

insert_to_table(data = data, table = table, database = database)

Añadidas: 895 filas


In [81]:
data = pd.read_csv("provincias.csv", index_col=0).values
table = "provincia"
database = "AEMET"

insert_to_table(data = data, table = table, database = database)

Añadidas: 53 filas


In [83]:
data = pd.read_csv("indicativo.csv", index_col=0).replace({np.nan : None}).values
table = "indicativo"
database = "AEMET"

insert_to_table(data = data, table = table, database = database)

Añadidas: 904 filas


In [85]:
data = pd.read_csv("df_22.csv", index_col=0).replace({np.nan : None}).values
table = "valores_climatologicos"
database = "AEMET"

insert_to_tabledos(data = data, table = table, database = database)

Columnas de la tabla: ('fecha', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velemedia', 'racha', 'horaracha', 'hrMedia', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'indicativo_id', 'nombre_id', 'provincia_id')
Ejemplo de fila de datos: [115 19.8 0.0 13.4 None 26.1 None 7.0 1.9 4.4 None 41.0 95.0 None 16.0
 None 1 1 1]
Error en los datos: El número de columnas y valores no coincide.


InternalError: Unread result found

In [ ]:
data = pd.read_csv("df_23.csv", index_col=0).replace({np.nan : None}).values
table = "valores_climatologicos"
database = "AEMET"

insert_to_table(data = data, table = table, database = database)

In [ ]:
data = pd.read_csv("df_24.csv", index_col=0).replace({np.nan : None}).values
table = "valores_climatologicos"
database = "AEMET"

insert_to_table(data = data, table = table, database = database)

In [29]:
def insert_to_tabledos(data, table, database, host="localhost", user="root", password="nueva_contraseña"):
    try:
        db = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        cursor = db.cursor()

        # Seleccionamos las columnas de la tabla, omitiendo la Primary Key
        cursor.execute(f"SELECT * FROM {table} LIMIT 0;")
        column_names = cursor.column_names[1:]  # Omitir la Primary Key

        # Verificar si el número de columnas coincide con el de valores
        print(f"Columnas de la tabla: {column_names}")
        print(f"Ejemplo de fila de datos: {data[0]}")
        if len(column_names) != len(data[0]):
            raise ValueError("El número de columnas y valores no coincide.")

        # Construcción del query de inserción
        insert_query = f"INSERT INTO {table} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"
        values = [tuple(row) for row in data]

        # Ejecutamos el query de inserción con executemany
        cursor.executemany(insert_query, values)
        
        # Confirmamos la transacción
        db.commit()

        print(f"Añadidas: {cursor.rowcount} filas")

    except mysql.connector.Error as err:
        print(f"Error en MySQL: {err}")
    except ValueError as ve:
        print(f"Error en los datos: {ve}")
    finally:
        cursor.close()
        db.close()

In [31]:
data = pd.read_csv("df_22.csv", index_col=0).replace({np.nan : None}).values
table = "valores_climatologicos"
database = "AEMET"

insert_to_tabledos(data = data, table = table, database = database)

Columnas de la tabla: ('fecha', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velemedia', 'racha', 'horaracha', 'hrMedia', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'indicativo_id', 'nombre_id', 'provincia_id')
Ejemplo de fila de datos: [115 19.8 0.0 13.4 None 26.1 None 7.0 1.9 4.4 None 41.0 95.0 None 16.0
 None 1 1 1]
Error en los datos: El número de columnas y valores no coincide.


InternalError: Unread result found